# Basic analysis

In [1]:
# import Bach
from bach_open_taxonomy import ObjectivFrame 

In [2]:
# connect to SQL db
of = ObjectivFrame.from_objectiv_data(db_url='postgresql://@localhost:5433/objectiv',
                                      table_name='data',
                                      time_aggregation='YYYY-MM-DD',
                                      start_date='2022-02-02')

In [3]:
# adding specific contexts to the data
of['application'] = of.global_contexts.gc.get_from_context_with_type_series(type='ApplicationContext', key='id')
of['root_location'] = of.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
of['referrer'] = of.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='referrer')
of['feature_nice_name'] = of.location_stack.ls.nice_name

In [4]:
# explore the data
of.sort_values('session_id', ascending=False).head()

,day,moment,user_id,global_contexts,location_stack,event_type,stack_event_types,session_id,session_hit_number,application,root_location,referrer,feature_nice_name
event_id,,,,,,,,,,,,,
81b19582-e106-4115-a0be-1165c2d6be0d,2022-02-16,2022-02-16 14:52:29.662,30ea7fc7-7dbe-4080-b980-55120503479d,"[{'id': 'http_context', '_type': 'HttpContext'...","[{'id': 'tracking', '_type': 'RootLocationCont...",ApplicationLoadedEvent,"[AbstractEvent, ApplicationLoadedEvent, NonInt...",1034,3,objectiv-docs,tracking,,Root Location: tracking
5623f530-81c1-425d-9b0e-1504f9b3170c,2022-02-16,2022-02-16 14:52:29.658,30ea7fc7-7dbe-4080-b980-55120503479d,"[{'id': 'http_context', '_type': 'HttpContext'...","[{'id': 'tracking', '_type': 'RootLocationCont...",VisibleEvent,"[AbstractEvent, NonInteractiveEvent, VisibleEv...",1034,2,objectiv-docs,tracking,,Expandable: Angular SDK located at Root Locati...
332f409d-cfc2-4cf3-81a8-3bde3054257c,2022-02-16,2022-02-16 14:52:29.658,30ea7fc7-7dbe-4080-b980-55120503479d,"[{'id': 'http_context', '_type': 'HttpContext'...","[{'id': 'tracking', '_type': 'RootLocationCont...",VisibleEvent,"[AbstractEvent, NonInteractiveEvent, VisibleEv...",1034,1,objectiv-docs,tracking,,Expandable: How-to-Guides located at Root Loca...
6d80c41e-b3da-4fd5-be64-d211afb0d78c,2022-02-16,2022-02-16 14:48:23.147,7c406acb-3d85-416f-a515-1caf6bc2a8a1,"[{'id': 'http_context', '_type': 'HttpContext'...","[{'id': 'tracking', '_type': 'RootLocationCont...",VisibleEvent,"[AbstractEvent, NonInteractiveEvent, VisibleEv...",1033,1,objectiv-docs,tracking,,Expandable: Angular SDK located at Root Locati...
b008f3bc-5c75-4241-a658-f19df13b59ab,2022-02-16,2022-02-16 14:48:23.147,7c406acb-3d85-416f-a515-1caf6bc2a8a1,"[{'id': 'http_context', '_type': 'HttpContext'...","[{'id': 'tracking', '_type': 'RootLocationCont...",VisibleEvent,"[AbstractEvent, NonInteractiveEvent, VisibleEv...",1033,2,objectiv-docs,tracking,,Expandable: How-to-Guides located at Root Loca...


In [5]:
# explore the data with describe
of.describe(include='all').head()

,day,moment,event_type,session_id,session_hit_number,application,root_location,referrer,feature_nice_name
__stat,,,,,,,,,
count,10039,10039,10039,10039.00,10039.00,10039,10039,1665,10039
mean,None,None,None,442.15,22.43,None,None,None,None
std,None,None,None,339.51,36.73,None,None,None,None
min,2022-02-02,2022-02-02 06:49:11.152,ApplicationLoadedEvent,1.00,1.00,objectiv-docs,about,,Expandable: Aggregation & windowing located at...
max,2022-02-16,2022-02-16 14:52:29.662,VisibleEvent,1034.00,308.00,objectiv-website,tracking,https://www.objectiv.io/,Root Location: tracking


## Users

In [6]:
# model hub: unique users, total month
users = of.model_hub.aggregate.unique_users(time_aggregation='YYYY-MM')
users.sort_index(ascending=False).head()

moment
2022-02    538
Name: unique_users, dtype: int64

In [7]:
# model hub: unique users, daily
users = of.model_hub.aggregate.unique_users()
users.sort_index(ascending=False).head(15)

moment
2022-02-16     19
2022-02-15     21
2022-02-14     28
2022-02-13      3
2022-02-12      8
2022-02-11     24
2022-02-10     16
2022-02-09     26
2022-02-08     35
2022-02-07     52
2022-02-06     21
2022-02-05     34
2022-02-04     58
2022-02-03    115
2022-02-02    245
Name: unique_users, dtype: int64

In [8]:
# users by RootLocation
users_root = of.groupby(['application', 'root_location']).agg({'user_id':'nunique'})
users_root.sort_values('user_id_nunique', ascending=False).head(10)

user_id_nunique
application      root_location                 
objectiv-website blog                       281
                 home                       274
objectiv-docs    docs                       175
objectiv-website about                       90
                 jobs                        79
objectiv-docs    home                        18
                 modeling                    12
                 tracking                    11
                 taxonomy                    10
objectiv-website privacy                      8

## Time spent

In [9]:
# model hub: duration, total month
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM')
duration.sort_index(ascending=False).head()

moment
2022-02   0 days 00:05:44.041577
Name: session_duration, dtype: timedelta64[ns]

In [10]:
# model hub: duration, daily
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM-DD')
duration.sort_index(ascending=False).head()

moment
2022-02-16   0 days 00:08:05.394577
2022-02-15   0 days 00:11:16.813353
2022-02-14   0 days 00:11:33.540325
2022-02-13   0 days 00:00:24.635000
2022-02-12   0 days 00:02:30.655000
Name: session_duration, dtype: timedelta64[ns]

In [11]:
# duration for RootLocation home 
of[(of.root_location == 'home')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:06:23.925473
Name: session_duration, dtype: timedelta64[ns]

In [12]:
# duration for RootLocation blog
of[(of.root_location == 'blog')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:02:58.632819
Name: session_duration, dtype: timedelta64[ns]

In [13]:
# duration for the whole docs
of[(of.application == 'objectiv-docs')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:08:03.566851
Name: session_duration, dtype: timedelta64[ns]

## Distribution of time spent

In [14]:
# first calculate the duration of each session
session_duration = of.groupby(['session_id']).agg({'moment':['min', 'max']})
session_duration['duration'] = session_duration['moment_max'] - session_duration['moment_min']

# explore the durations with describe
session_duration['duration'].describe().head()

__stat
count                 1034
mean       00:04:55.463172
min               00:00:00
max            06:03:47.69
nunique                843
Name: duration, dtype: object

In [15]:
# materizalize the df
session_duration.materialize(inplace=True)

# define the quantiles 
quantiles = [0.25, 0.25, 0.75]

# distribution of time spemt
session_duration['duration'].quantile(q=quantiles).head()

q
0.25   0 days 00:00:00.682000
0.25   0 days 00:00:00.682000
0.75   0 days 00:03:05.253500
Name: duration_duration_duration, dtype: timedelta64[ns]

## Top user interactions

In [16]:
# select PressEvent and MediaStart event, to focus on user actions
users_feature = of[(of.event_type == 'PressEvent') |
                   (of.event_type == 'MediaStartEvent')]

# users by feature
users_feature = users_feature.groupby(['application', 'feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
users_feature.sort_values('user_id_nunique', ascending=False).head()

user_id_nunique
application      feature_nice_name                                  event_type                      
objectiv-website Media Player: objectiv-in-2-minutes located at ... MediaStartEvent              104
                 Media Player: 2-minute-video located at Root Lo... MediaStartEvent               78
                 Link: about-us located at Root Location: home =... PressEvent                    53
                 Pressable: hamburger located at Root Location: ... PressEvent                    43
                 Link: logo located at Root Location: blog => Na... PressEvent                    38

## What did users do most on home?

In [17]:
# select homepage RootLocation only
home_users = of[(of.root_location == 'home')]

# select website only and exclude the docs
home_users = home_users[(home_users.application == 'objectiv-website')]

# select PressEvent and MediaStart event, to focus on user actions
home_users = home_users[(home_users.event_type == 'PressEvent') |
                   (home_users.event_type == 'MediaStartEvent')]

# home users by feature
home_users = home_users.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
home_users.sort_values('user_id_nunique', ascending=False).head()

,,user_id_nunique
feature_nice_name,event_type,
Media Player: 2-minute-video located at Root Location: home,MediaStartEvent,78
Link: about-us located at Root Location: home => Navigation: navbar-top,PressEvent,53
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,PressEvent,43
Link: docs located at Root Location: home => Navigation: navbar-top,PressEvent,24
Link: blog located at Root Location: home => Navigation: navbar-top,PressEvent,23


## What did users do most in docs?

In [18]:
# select docs RootLocation only
docs_users = of[(of.application == 'objectiv-docs')]

# select PressEvent and MediaStart event, to focus on user actions
docs_users = docs_users[(docs_users.event_type == 'PressEvent') |
                   (docs_users.event_type == 'MediaStartEvent')]

# docs users by feature
docs_users = docs_users.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
docs_users.sort_values('user_id_nunique', ascending=False).head()

,,user_id_nunique
feature_nice_name,event_type,
Link: Modeling located at Root Location: docs => Navigation: navbar-top,PressEvent,25
Link: Tracking located at Root Location: docs => Navigation: navbar-top,PressEvent,24
Link: logo located at Root Location: docs => Navigation: navbar-top,PressEvent,23
Link: Taxonomy located at Root Location: docs => Navigation: navbar-top,PressEvent,18
Link: Core Concepts located at Root Location: docs => Navigation: docs-sidebar,PressEvent,14


## Where did users come from?

In [19]:
# users by referrer
referrer_users = of.groupby(['referrer']).agg({'user_id':'nunique'})
referrer_users.sort_values('user_id_nunique', ascending=False).head()

,user_id_nunique
referrer,
None,482
,61
https://www.google.com/,5
https://t.co/,4
https://t.co/4EqHiQfiOf,3


## Set conversion

In [20]:
# define which events to use as conversion events
of.add_conversion_event(location_stack=of.location_stack.json[{'id': 'objectiv-on-github', 
                                                        '_type': 'LinkContext'}:].fillna(
                                       of.location_stack.json[{'id': 'github', '_type': 'LinkContext'}:]),
                        event_type='PressEvent',
                        name='github_press')

## Number of daily conversions

In [21]:
of.model_hub.filter(of.model_hub.map.is_conversion_event('github_press'))\
    .model_hub.aggregate.unique_users()\
    .sort_index(ascending=False).head(10)

moment
2022-02-16    2
2022-02-14    1
2022-02-13    1
2022-02-12    1
2022-02-11    2
2022-02-10    2
2022-02-09    3
2022-02-07    3
2022-02-04    5
2022-02-03    7
Name: unique_users, dtype: int64

## From where do users convert most?

In [22]:
# filter conversion events
conversion_locations = of.model_hub.filter(of.model_hub.map.is_conversion_event('github_press'))\
    .groupby(['application', 'feature_nice_name', 'event_type'])\
    .agg({'user_id':'nunique'})

conversion_locations.sort_values('user_id_nunique', ascending=False).head()

user_id_nunique
application      feature_nice_name                                  event_type                 
objectiv-website Link: objectiv-on-github located at Root Locati... PressEvent               20
                 Link: objectiv-on-github located at Root Locati... PressEvent               12
                 Link: github located at Root Location: home => ... PressEvent                8
                 Link: github located at Root Location: blog => ... PressEvent                2
                 Link: github located at Root Location: home => ... PressEvent                2

## Time spent for converting users?

In [23]:
# use model hub to filter converting users and calculate duration
of.model_hub.filter(of.model_hub.map.conversions_counter(name='github_press', partition='user_id')>=1)\
  .model_hub.agg.session_duration()\
  .sort_index(ascending=False).head()

moment
2022-02-16   0 days 00:11:50.661182
2022-02-15   0 days 00:11:02.535125
2022-02-14   0 days 00:25:53.118500
2022-02-13   0 days 00:00:06.779000
2022-02-12   0 days 00:03:10.018000
Name: session_duration, dtype: timedelta64[ns]

## Top user interactions before conversions

In [24]:
# select sessions with a conversion
converted_users = of.model_hub.filter(of.model_hub.map.conversions_counter(name='github_press')>=1)

# from those, select hits where number of conversions was still 0
converted_users = converted_users.mh.filter(converted_users.model_hub.map.conversions_in_time('github_press')==0)

# select PressEvent and MediaStart event, to focus on user actions
converted_users = converted_users[(converted_users.event_type == 'PressEvent') |
                   (converted_users.event_type == 'MediaStartEvent')]

converted_users.groupby(['application', 'feature_nice_name', 'event_type']).agg({'user_id':'nunique'})\
    .sort_values('user_id_nunique', ascending=False).head(10)

user_id_nunique
application      feature_nice_name                                  event_type                      
objectiv-website Media Player: 2-minute-video located at Root Lo... MediaStartEvent                6
                 Link: logo located at Root Location: blog => Na... PressEvent                     5
                 Media Player: objectiv-in-2-minutes located at ... MediaStartEvent                5
                 Link: blog located at Root Location: home => Na... PressEvent                     4
                 Link: meet-objectiv-open-source-product-analyti... PressEvent                     3
                 Link: faq located at Root Location: home => Nav... PressEvent                     3
                 Pressable: hamburger located at Root Location: ... PressEvent                     3
                 Link: jobs located at Root Location: blog => Na... PressEvent                     2
objectiv-docs    Link: HttpContext located at Root Location: doc... PressEvent                     2
objectiv-website Link: docs located at Root Location: home => Na... PressEvent                     2

## Time spent before conversions

In [25]:
# use model hub to calculate duration, converted users contains events before 1st conversion
converted_users.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:04:47.335706
Name: session_duration, dtype: timedelta64[ns]